In [27]:
import numpy as np
import datetime, time

src_filename = '../Datasets/kkbox/churn/raw/user_logs.csv'
dst_template = '../Datasets/kkbox/churn/raw/user_logs/uldt_{0}.csv'

dst_years = ['2015','2016','2017']
dst_months = ['{0:02}'.format(i+1) for i in range(12)]
dst_files = [z for z in [x+y for x in dst_years for y in dst_months] if z <= '201702']

['201501',
 '201502',
 '201503',
 '201504',
 '201505',
 '201506',
 '201507',
 '201508',
 '201509',
 '201510',
 '201511',
 '201512',
 '201601',
 '201602',
 '201603',
 '201604',
 '201605',
 '201606',
 '201607',
 '201608',
 '201609',
 '201610',
 '201611',
 '201612',
 '201701',
 '201702']

In [39]:
batch_size = 10000000 #1mln
num_steps = 1000
header = None
with open(src_filename,'r') as hf:
    header = hf.readline()
    
dst_fs = {x:open(dst_template.format(x),'w') for x in dst_files}
for p in dst_fs:
    dst_fs[p].writelines(header)
print_fmt = '{0}/{1} steps complete; {2:.2f} seconds (I:{3:.2f}, S:{4:.2f}, T:{5:.2f}, O:{6:.2f})'

with open(src_filename,'r') as srcf:
    srcf.readline() #skip first
    for step in range(num_steps):
        t0 = time.perf_counter()
        lines = [srcf.readline().split(',') for i in range(batch_size)]
        if lines is None or len(lines)<=0:
            break
        t1 = time.perf_counter()
        lines.sort(key=lambda x:x[1])
        t2 = time.perf_counter()
        sublines = [[0,len(lines)]]
        kstr = lines[0][1][:6]
        for i in range(1, len(lines)):
            if lines[i][1][:6]==kstr:
                continue
            sublines[-1] = [sublines[-1][0], i]
            sublines.append([i, len(lines)])
            kstr = lines[i][1][:6]
        t3 = time.perf_counter()
        for p in sublines:
            df = dst_fs[lines[p[0]][1][:6]]
            df.writelines([','.join(lines[x]) for x in range(p[0],p[1])])
            df.flush()
        t4 = time.perf_counter()
        
        print(print_fmt.format(step+1, num_steps, t4-t0, t1-t0, t2-t1,t3-t2,t4-t3), end='\r')

IndexError: list index out of range